In [ ]:
%pip install pyautogen
%pip install langchain
%pip install langchain_community
%pip install requests
%pip install beautifulsoup4
%pip install markdownify
%pip install --upgrade --quiet  youtube-transcript-api
%pip install "pyautogen>=0.2.18" "tavily-python"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.6/284.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import userdata
llm_config=[{"model": "gpt-4o", "api_key":userdata.get('open_api_key')}]
llm_config2=[{"model": "gpt-3.5-turbo", "api_key":userdata.get('open_api_key')}]

### Function Callings and Prompt Engineering

In [ ]:
from typing import Annotated
from langchain_community.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tavily import TavilyClient
import requests
from bs4 import BeautifulSoup
from markdownify import markdownify

tavily = TavilyClient(api_key=userdata.get('talivy_key'))
def search_tool(query: Annotated[str, "The search query"]) -> Annotated[str, "The search results"]:
  return tavily.get_search_context(query=query, search_depth="advanced")

def youtube_transcript_retrieval(url):
  loader = YoutubeLoader.from_youtube_url(url)
  docs = loader.load()
  docs= ' '.join([doc.page_content for doc in docs])
  return docs

def content_retrieval(url):
  try:
    responses = requests.get(url)
  except:
    print("Errors occured in retrieving the data")
  soup = BeautifulSoup(responses.text, 'html.parser')
  texts=soup.get_text()
  return markdownify(texts)

def split_text(docs):
  text_splitters=RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
  docs=text_splitters.create_documents(docs)
  return docs

### AI Agents

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from autogen import UserProxyAgent, AssistantAgent, register_function
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.coding import LocalCommandLineCodeExecutor


code_executor=LocalCommandLineCodeExecutor(work_dir="/content/sample_data")

researcher = AssistantAgent(
    name="Researcher",
    llm_config={"config_list":llm_config2},
    human_input_mode="NEVER",
    system_message="You are a researcher tasked with summarizing notable information from a given context. Your role is to analyze the provided research content related to a topic. You should concisely summarize the 5 most relevant insights, facts, in bullet points while remaining unbiased. Your goal is to provide notable pơints of the content to facilitate further validation."
)

questioner = AssistantAgent(
    name="questioner",
    llm_config={"config_list":llm_config},
    system_message="You are a critical thinker tasked for assessing the credibility of information from various angles."
    "Your role is to carefully analyze the information summarized by the Researcher, and validate the quality as well as trustworthiness of that information."
    "You should assess every piece of insight by questioning its logic, accuracy and biases."
    "You have access to tools provided. Use the following format:"
    "Information: the insights that need to be assessed"
    "Thought: you should always think about what to do"
    "Action: the action to take"
    "Action Input: the input to the action"
    "Observation: the result of the action"
    "... (this process can repeat multiple times)"
    "Thought: I now know the final argument"
    "Final Answer: return the trustworthiness score, quality score of that information and reasons for that"
)


user_proxy=UserProxyAgent(
    name="SearchTool",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"executor": code_executor},
)


register_function(
    search_tool,
    caller=questioner,
    executor=user_proxy,
    name="search_tool",
    description="Search the web for the given query",
)

/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:2410: UserWarning: Function 'search_tool' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


### Test

In [ ]:
youtube_docs=youtube_transcript_retrieval("https://youtu.be/gj-3Ji3CN80?si=K4VeVCxjrYJXBKrF")

In [ ]:
task="This is the youtube transcript that I retrieve from a debate video: /n" + youtube_docs

In [ ]:
def reflection_message(recipient, messages, sender, config):
    print("Reflecting...", "yellow")
    return f"Validate all important information. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"

user_proxy.register_nested_chats(
  [
      {
          'recipient': questioner,
          "message": reflection_message,
          "summary_method": "last_msg",
          "max_turns": 2
      }
  ],
  trigger= researcher
)

In [ ]:
res = user_proxy.initiate_chat(recipient=researcher, message=task, max_turns=2, summary_method="last_msg")

User (to Researcher):

This is the youtube transcript that I retrieve from a debate video: /nyou're both mostly in agreement but what is I don't know if i' say that okay but as I'm learning uh what is Israel doing right what is Israel doing wrong in this very specific current war in Gaza um I mean frankly I think that what Israel is doing wrong is if I were Israel okay like again America's interests are not coincident with Israel's interest if if if I were an Israeli leader I would have swiveled up and and knocked the leap out ofah early does does mean so I I would have I would y Galant who was the defense minister of Israel was encouraging Netanyahu is the prime minister and the war cabinet including Benny Gan so whenever people talk about the Netanyahu government that's not what's in place right now there's a Unity War government in place that includes the political opposition the reason I point that out is because there are a lot of people politically who will suggest that the actio